# Similar Abstract Search

https://medium.com/@davidlfliang/intro-getting-started-with-text-embeddings-using-bert-9f8c3b98dee6

In [ ]:
# Use kaggle API to download example data. arxiv apaper Abstracts and there is also arXive Dataset (4.5 GB)
import kagglehub

# Download latest version
# path = kagglehub.dataset_download("Cornell-University/arxiv")

# print("data/arxiv_dataset", path)

/home/piotr/projects/articles_similarity/scientific_abstracts_similarity/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.37G/1.37G [04:39<00:00, 5.25MB/s]

Extracting files...


data/arxiv_dataset /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/208


In [ ]:
# !cp /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/208/arxiv-metadata-oai-snapshot.json /home/piotr/projects/articles_similarity/scientific_abstracts_similarity/data/arxiv_papers

## Preprocessing

### Tokenization

In [14]:
MODEL_NAME = 'bert-large-cased'

In [ ]:
# from transformers import BertTokenizer

# bert_tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# text = "The quick brown fox jumps over the lazy dog."

# bert_inputs = bert_tokenizer(text, return_tensors='pt')

# print('Token IDs: ', bert_inputs['input_ids'])

# attention_mask = bert_inputs['attention_mask']
# print('Attention Mask: ', attention_mask)

# token_type_ids = bert_inputs['token_type_ids']
# print('Token Type IDs:', token_type_ids)

# tokens = bert_tokenizer.convert_ids_to_tokens(bert_inputs['input_ids'][0])
# print('Tokens:', tokens)

Token IDs:  tensor([[  101,  1109,  3613,  3058, 17594, 15457,  1166,  1103, 16688,  3676,
           119,   102]])
Attention Mask:  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Token Type IDs: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Tokens: ['[CLS]', 'The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '[SEP]']


In [ ]:
# bert_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

## Embedding Generation

In [ ]:
# from transformers import BertModel
# import torch

# model = BertModel.from_pretrained(MODEL_NAME)

# with torch.no_grad():
#     outputs = model(**bert_inputs)

# last_hidden_states = outputs.last_hidden_state

# print('Shape of the last hidden state (embedding):', last_hidden_states.shape)

# tokens = bert_tokenizer.convert_ids_to_tokens(bert_inputs['input_ids'][0])
# for token, embedding in zip(tokens, last_hidden_states[0]):
#     print(f"Token: {token}, Embedding Dimension:{embedding.shape},\nEmbedding (first 5 components): {embedding[:5]}")


Shape of the last hidden state (embedding): torch.Size([1, 12, 1024])
Token: [CLS], Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.3982, -0.5164,  0.1184,  0.2094, -0.0663])
Token: The, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.5459, -0.9618,  0.1409, -0.1374, -0.1215])
Token: quick, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.1578,  0.3739, -0.2527, -0.0500,  0.7173])
Token: brown, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-1.0575, -0.4547,  0.0686, -0.6760,  0.7317])
Token: fox, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.1732, -0.2297, -0.7842,  0.0217,  0.0425])
Token: jumps, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.5367,  0.0404, -0.0642, -0.2768,  0.2793])
Token: over, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-

Models like SBERT (Sentence-BERT) are specialized in generating high-quality sentence embeddings.

## Text Similarity Search

### Handle 4.5 GB JSON file

In [8]:
MODEL_NAME = 'bert-large-cased'
dataset_path = '../data/arxiv_papers/arxiv-metadata-oai-snapshot.json'

Read one example line

In [9]:
import json

# Open the NDJSON file
with open(dataset_path, 'r') as f:
    # Read the first line
    line = f.readline().strip()
    # Parse the JSON object
    data = json.loads(line)
    print(data)  # Output the parsed JSON


{'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'report-no': 'ANL-HEP-PR-07-12', 'categories': 'hep-ph', 'license': None, 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab

Read lines from the file.

In [11]:
texts = []
with open(dataset_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        texts.append(data)
        print(data)
        if len(texts) > 9:
            break

{'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'report-no': 'ANL-HEP-PR-07-12', 'categories': 'hep-ph', 'license': None, 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab

## Create Embeddings

In [13]:
texts[3]

{'id': '0704.0004',
 'submitter': 'David Callan',
 'authors': 'David Callan',
 'title': 'A determinant of Stirling cycle numbers counts unlabeled acyclic\n  single-source automata',
 'comments': '11 pages',
 'journal-ref': None,
 'doi': None,
 'report-no': None,
 'categories': 'math.CO',
 'license': None,
 'abstract': '  We show that a determinant of Stirling cycle numbers counts unlabeled acyclic\nsingle-source automata. The proof involves a bijection from these automata to\ncertain marked lattice paths and a sign-reversing involution to evaluate the\ndeterminant.\n',
 'versions': [{'version': 'v1', 'created': 'Sat, 31 Mar 2007 03:16:14 GMT'}],
 'update_date': '2007-05-23',
 'authors_parsed': [['Callan', 'David', '']]}

In [15]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = torch.mean(last_hidden_states, dim=1).numpy()
    return sentence_embedding

In [16]:
embeddings = [get_sentence_embedding(text['abstract']) for text in texts]

In [20]:
query_embedding = embeddings[0]

In [21]:
similarities = cosine_similarity(query_embedding, np.vstack(embeddings[1:]))

In [57]:
print(sorted(similarities[0], reverse=True))


[np.float32(0.9335567), np.float32(0.88754284), np.float32(0.8645277), np.float32(0.84983355), np.float32(0.83770466), np.float32(0.788833), np.float32(0.7758506), np.float32(0.7589118), np.float32(0.7367904)]


In [53]:
# Print query text
print (f"Query text: {texts[0]['title']}")

# Print similarities
for i, text in enumerate(texts[1:]):
    print(f"Similarity with '{text['title']}': {similarities[0][i]}")

Query text: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies
Similarity with 'Sparsity-certifying Graph Decompositions': 0.7367904186248779
Similarity with 'The evolution of the Earth-Moon system based on the dark matter field
  fluid model': 0.7589117884635925
Similarity with 'A determinant of Stirling cycle numbers counts unlabeled acyclic
  single-source automata': 0.7758505940437317
Similarity with 'From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\alpha}$': 0.7888330221176147
Similarity with 'Bosonic characters of atomic Cooper pairs across resonance': 0.8377046585083008
Similarity with 'Polymer Quantum Mechanics and its Continuum Limit': 0.8498335480690002
Similarity with 'Numerical solution of shock and ramp compression for general material
  properties': 0.864527702331543
Similarity with 'The Spitzer c2d Survey of Large, Nearby, Insterstellar Clouds. IX. The
  Serpens YSO Population As Observed With IRAC and MIPS': 0.8875428438186646
Similarity